In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
import os

env_path = '../../data/.env'
load_dotenv(dotenv_path=env_path)

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

In [5]:
project = hopsworks.login(project='AMomozZz')
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

location_str, today

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163416
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


('{"country": "china", "city": "shanghai", "street": "jinganjiancezhan", "aqicn_url": "https://api.waqi.info/feed/@486", "latitude": 31.23, "longitude": 121.47}',
 datetime.date(2024, 11, 10))

In [6]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='new_aq',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='new_w',
    version=1,
)

In [7]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
display(aq_today_df)
aq_today_df.info()

,pm25,country,city,street,date,url
0,127.0,china,shanghai,jinganjiancezhan,2024-11-10,https://api.waqi.info/feed/@486


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


In [ ]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city

# yesterday's pm25 value
df_temp = air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(3)
df_temp['date'] = pd.to_datetime(df_temp['date']) + pd.Timedelta(days=2)
df_temp['date'] = pd.to_datetime(df_temp['date']).dt.tz_localize(None)
# add pm25 value from yesterday
daily_df = pd.merge(daily_df, df_temp[['date','pm25']], how='left', on="date")
daily_df = daily_df.rename(columns={"pm25": "pm25_yesterday"})

# today's pm25 value
pm25_value = aq_today_df.iloc[0]['pm25']
today_date = aq_today_df.iloc[0]['date']
next_day = today_date + pd.Timedelta(days=1)
condition = (daily_df['date'] == next_day)
# add pm25 value from today
daily_df.loc[condition, 'pm25_yesterday'] = pm25_value

display(daily_df)
daily_df.info()

Coordinates 31.25°N 121.5°E
Elevation 3.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.61s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_tdby
0,2024-11-09,17.950001,0.0,13.392774,36.253937,shanghai,35.0
1,2024-11-10,15.400000,0.0,13.320000,360.000000,shanghai,55.0
2,2024-11-11,15.250000,0.0,10.195057,42.137531,shanghai,46.0
3,2024-11-12,15.550000,0.0,9.339208,62.447273,shanghai,127.0
4,2024-11-13,16.250000,0.0,11.013882,101.309898,shanghai,NaN
5,2024-11-14,20.150000,0.0,9.693296,148.671371,shanghai,NaN
6,2024-11-15,20.450001,0.1,5.191994,56.309914,shanghai,NaN
7,2024-11-16,19.799999,0.2,7.421590,22.833694,shanghai,NaN
8,2024-11-17,15.500000,0.0,24.336637,352.349426,shanghai,NaN
9,2024-11-18,10.850000,0.0,15.725037,15.945477,shanghai,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         11 non-null     datetime64[ns]
 1   temperature_2m_mean          11 non-null     float32       
 2   precipitation_sum            11 non-null     float32       
 3   wind_speed_10m_max           11 non-null     float32       
 4   wind_direction_10m_dominant  11 non-null     float32       
 5   city                         11 non-null     object        
 6   pm25_tdby                    4 non-null      float32       
dtypes: datetime64[ns](1), float32(5), object(1)
memory usage: 528.0+ bytes


In [9]:
display(air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(5))
display(weather_fg.read().sort_values(by=['date'], ascending=True).tail(15))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.12s) 


,date,pm25,country,city,street,url
1286,2024-11-06 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1370,2024-11-07 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3121,2024-11-08 00:00:00+00:00,55.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
2555,2024-11-09 00:00:00+00:00,46.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3039,2024-11-10 00:00:00+00:00,114.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.88s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_tdby
3610,2024-10-25 00:00:00+00:00,22.088249,1.200000,27.128096,50.183929,shanghai,95.0
2421,2024-10-26 00:00:00+00:00,22.473669,11.800001,20.721901,76.058823,shanghai,46.0
2116,2024-10-27 00:00:00+00:00,18.394499,4.600000,26.525429,355.428314,shanghai,47.0
1528,2024-10-28 00:00:00+00:00,15.811168,0.000000,19.320465,347.887512,shanghai,23.0
59,2024-10-29 00:00:00+00:00,16.359083,0.000000,16.622490,1.761852,shanghai,34.0
1453,2024-10-30 00:00:00+00:00,19.104918,0.000000,23.185787,42.530087,shanghai,115.0
285,2024-10-31 00:00:00+00:00,21.023664,72.799995,24.979176,44.617844,shanghai,49.0
2249,2024-11-01 00:00:00+00:00,18.234087,58.000004,41.148014,1.924738,shanghai,54.0
431,2024-11-02 00:00:00+00:00,17.463247,0.000000,24.215185,335.282562,shanghai,49.0
2330,2024-11-03 00:00:00+00:00,17.740332,0.000000,8.149847,16.130766,shanghai,20.0


In [10]:
# Insert new data
_ = air_quality_fg.insert(aq_today_df)

2024-11-10 21:22:55,310 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1344871


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: new_aq_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163416/jobs/named/new_aq_1_offline_fg_materialization/executions


In [11]:
# Insert new data
_ = weather_fg.insert(daily_df)

2024-11-10 21:23:05,282 INFO: 	3 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1344872


Uploading Dataframe: 0.00% |          | Rows 0/11 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: new_w_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163416/jobs/named/new_w_1_offline_fg_materialization/executions


In [15]:
display(air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(5))
display(weather_fg.read().sort_values(by=['date'], ascending=True).tail(15))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.11s) 


,date,pm25,country,city,street,url
1286,2024-11-06 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1370,2024-11-07 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3121,2024-11-08 00:00:00+00:00,55.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
2555,2024-11-09 00:00:00+00:00,46.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3039,2024-11-10 00:00:00+00:00,127.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.71s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_tdby
2921,2024-11-05 00:00:00+00:00,13.817414,0.0,19.852295,358.205353,shanghai,140.0
1253,2024-11-06 00:00:00+00:00,13.725750,0.0,17.589418,15.969136,shanghai,124.0
2532,2024-11-07 00:00:00+00:00,14.234082,0.0,14.450535,34.656887,shanghai,45.0
2318,2024-11-08 00:00:00+00:00,17.711164,0.0,18.275984,44.213470,shanghai,35.0
3954,2024-11-09 00:00:00+00:00,17.950001,0.0,13.392774,36.253937,shanghai,35.0
3950,2024-11-10 00:00:00+00:00,15.400000,0.0,13.320000,360.000000,shanghai,55.0
3951,2024-11-11 00:00:00+00:00,15.250000,0.0,10.195057,42.137531,shanghai,46.0
3956,2024-11-12 00:00:00+00:00,15.550000,0.0,9.339208,62.447273,shanghai,127.0
3958,2024-11-13 00:00:00+00:00,16.250000,0.0,11.013882,101.309898,shanghai,NaN
3952,2024-11-14 00:00:00+00:00,20.150000,0.0,9.693296,148.671371,shanghai,NaN
